<a href="https://colab.research.google.com/github/Mbaiornom/Essai/blob/main/filtre_LEON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np

raw_bva = pd.read_csv('../Données brutes/Baro_SNCF_2023_Data_2023.csv', sep=";", index_col="ID BDD")
raw_bva.head(5)

raw_bva['L1'] = raw_bva['L1'].fillna(2).astype(int)
raw_bva['L2'] = raw_bva['L2'].fillna(9).astype(int)
raw_bva['L3'] = raw_bva['L3'].fillna(38).astype(int)
bva = raw_bva.filter(regex='^CLASSE$|^L1$|^L2$|^L3$')
bva.head()

,CLASSE,L1,L2,L3
ID BDD,,,,
1,4,1,5,32
2,2,1,5,34
3,4,1,5,32
4,5,1,5,34
5,2,1,5,32


# Etablissements

In [ ]:
bva_etablissement = raw_bva.filter(regex='^ETABLISSEMENT_')

def find_etablissement(row):
    etablissements = row[row == 1].index.tolist()[:3]
    res = {'ets_ref': etablissements[-1] if len(etablissements) != 0 else None}
    if len(etablissements) < 3:
        etablissements += [None] * (3 - len(etablissements))
    return {**res, **{'ets1': etablissements[0], 'ets2': etablissements[1], 'ets3': etablissements[2]}}

bva_etablissement = bva_etablissement.apply(find_etablissement, axis=1).apply(pd.Series)
bva_etablissement = bva_etablissement.drop(columns=bva_etablissement.columns.difference(['ets_ref','ets1', 'ets2', 'ets3']))
bva_etablissement['ets_ref'] = bva_etablissement['ets_ref'].str.replace('ETABLISSEMENT_', '')
bva_etablissement['ets1'] = bva_etablissement['ets1'].str.replace('ETABLISSEMENT_', '')
bva_etablissement['ets2'] = bva_etablissement['ets2'].str.replace('ETABLISSEMENT_', '')
bva_etablissement['ets3'] = bva_etablissement['ets3'].str.replace('ETABLISSEMENT_', '')
bva[['ets_ref','ets1','ets2','ets3']] = bva_etablissement[['ets_ref','ets1','ets2','ets3']]
bva.head()

C:\Users\0104785X\AppData\Local\Temp\ipykernel_21328\3010722020.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bva[['ets_ref','ets1','ets2','ets3']] = bva_etablissement[['ets_ref','ets1','ets2','ets3']]
C:\Users\0104785X\AppData\Local\Temp\ipykernel_21328\3010722020.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bva[['ets_ref','ets1','ets2','ets3']] = bva_etablissement[['ets_ref','ets1','ets2','ets3']]
C:\Users\0104785X\AppData\Local\Temp\ipykernel_21328\3010722020.py:16: SettingWithCopyWarning:

,CLASSE,L1,L2,L3,ets_ref,ets1,ets2,ets3
0,4,1,5,32,317,317,None,None
1,4,1,5,32,318,318,None,None
2,1,1,5,34,187,68,187,None
3,4,1,5,32,200,200,None,None
4,4,1,5,33,241,87,241,None


# Engagement

### Standard

In [ ]:
bva_engagement_std = raw_bva.filter(regex='^Q2$|^Q3$|^Q4$|^Q5$|^Q13$')

def calculate_standard(x):
    if x == 1 or x == 2:
        return 1
    elif x == 3 or x == 4:
        return 0
    elif x == 5:
        return np.nan
    else:
        return x

bva_engagement_std = bva_engagement_std.map(calculate_standard)
bva_engagement_std.head()

,Q2,Q3,Q4,Q5,Q13
0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,1.0
2,1.0,1.0,1.0,1.0,1.0
3,1.0,1.0,0.0,0.0,1.0
4,0.0,0.0,0.0,0.0,1.0


### Detail

In [ ]:
bva_engagement_dtl = raw_bva.filter(regex='^Q2$|^Q3$|^Q4$|^Q5$|^Q13$')

def calculate_detail(x):
    if x == 1 :
        return 1
    elif x == 2:
        return 0.67
    elif x == 3:
        return 0.33
    elif x == 4:
        return 0
    elif x == 5:
        return np.nan
    else:
        return x

bva_engagement_dtl = bva_engagement_dtl.map(calculate_detail)
bva_engagement_dtl.head()

,Q2,Q3,Q4,Q5,Q13
0,0.33,0.33,0.0,0.00,0.33
1,0.67,0.33,0.0,0.00,1.00
2,1.00,1.00,1.0,1.00,1.00
3,0.67,0.67,0.0,0.00,1.00
4,0.33,0.33,0.0,0.33,0.67


In [ ]:
bva['Engagement standard'] = bva_engagement_std[['Q2','Q3','Q4', 'Q5', 'Q13']].mean(axis=1, skipna=True)
bva['Engagement detail'] = bva_engagement_dtl[['Q2','Q3','Q4', 'Q5', 'Q13']].mean(axis=1, skipna=True)
bva = pd.concat([bva, bva_engagement_std], axis=1)
bva.head()

C:\Users\0104785X\AppData\Local\Temp\ipykernel_21328\2966444818.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bva['Engagement standard'] = bva_engagement_std[['Q2','Q3','Q4', 'Q5', 'Q13']].mean(axis=1, skipna=True)
C:\Users\0104785X\AppData\Local\Temp\ipykernel_21328\2966444818.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bva['Engagement detail'] = bva_engagement_dtl[['Q2','Q3','Q4', 'Q5', 'Q13']].mean(axis=1, skipna=True)


,CLASSE,L1,L2,L3,ets_ref,ets1,ets2,ets3,Engagement standard,Engagement detail,Q2,Q3,Q4,Q5,Q13
0,4,1,5,32,317,317,None,None,0.0,0.198,0.0,0.0,0.0,0.0,0.0
1,4,1,5,32,318,318,None,None,0.4,0.400,1.0,0.0,0.0,0.0,1.0
2,1,1,5,34,187,68,187,None,1.0,1.000,1.0,1.0,1.0,1.0,1.0
3,4,1,5,32,200,200,None,None,0.6,0.468,1.0,1.0,0.0,0.0,1.0
4,4,1,5,33,241,87,241,None,0.2,0.332,0.0,0.0,0.0,0.0,1.0


# QVT

### Standard

In [ ]:
bva_qvt_std = raw_bva.filter(regex='^Q6$|^Q7$|^Q8$|^Q9_|^Q10$')
bva_qvt_std = bva_qvt_std.map(calculate_standard)
bva_qvt_std['Q9'] = bva_qvt_std.filter(regex='^Q9_').mean(axis=1, skipna=True)
bva['QVT standard'] = bva_qvt_std[['Q6','Q7','Q8', 'Q9', 'Q10']].mean(axis=1, skipna=True)
bva = pd.concat([bva, bva_qvt_std], axis=1)
bva.head()

,CLASSE,L1,L2,L3,ets_ref,ets1,ets2,ets3,Engagement standard,Engagement detail,...,Q9_1_1,Q9_2_2,Q9_3_3,Q9_4_4,Q9_5_5,Q9_6_6,Q9_7_7,Q9_8_8,Q10,Q9
0,4,1,5,32,317,317,None,None,0.0,0.198,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000
1,4,1,5,32,318,318,None,None,0.4,0.400,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.375
2,1,1,5,34,187,68,187,None,1.0,1.000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000
3,4,1,5,32,200,200,None,None,0.6,0.468,...,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.375
4,4,1,5,33,241,87,241,None,0.2,0.332,...,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.500


### Detail

In [ ]:
bva_qvt_dtl = raw_bva.filter(regex='^Q6$|^Q7$|^Q8$|^Q9_|^Q10$|^Q9_')
bva_qvt_dtl = bva_qvt_dtl.map(calculate_detail)
bva_qvt_dtl['Q9'] = bva_qvt_dtl.filter(regex='^Q9_').mean(axis=1, skipna=True)
bva['QVT detail'] = bva_qvt_dtl[['Q6','Q7','Q8', 'Q9', 'Q10']].mean(axis=1, skipna=True)
bva.head()

,CLASSE,L1,L2,L3,ets_ref,ets1,ets2,ets3,Engagement standard,Engagement detail,...,Q9_2_2,Q9_3_3,Q9_4_4,Q9_5_5,Q9_6_6,Q9_7_7,Q9_8_8,Q10,Q9,QVT detail
0,4,1,5,32,317,317,None,None,0.0,0.198,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.13200
1,4,1,5,32,318,318,None,None,0.4,0.400,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.375,0.59450
2,1,1,5,34,187,68,187,None,1.0,1.000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.000,0.86800
3,4,1,5,32,200,200,None,None,0.6,0.468,...,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.375,0.33275
4,4,1,5,33,241,87,241,None,0.2,0.332,...,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.500,0.35775


In [ ]:
bva[['Q6','Q7','Q8', 'Q9', 'Q10']] = bva_qvt_std[['Q6','Q7','Q8', 'Q9', 'Q10']]

# Q10A et Q29

In [ ]:
bva['Q29'] = raw_bva['Q29'].apply(calculate_standard)
bva['Q10A'] = raw_bva['Q10A'].apply(calculate_standard)
bva.head()

,CLASSE,L1,L2,L3,ets_ref,ets1,ets2,ets3,Engagement standard,Engagement detail,...,Q9_4_4,Q9_5_5,Q9_6_6,Q9_7_7,Q9_8_8,Q10,Q9,QVT detail,Q29,Q10A
0,4,1,5,32,317,317,None,None,0.0,0.198,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.13200,0.0,0.0
1,4,1,5,32,318,318,None,None,0.4,0.400,...,0.0,1.0,1.0,1.0,0.0,1.0,0.375,0.59450,0.0,0.0
2,1,1,5,34,187,68,187,None,1.0,1.000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.000,0.86800,1.0,1.0
3,4,1,5,32,200,200,None,None,0.6,0.468,...,1.0,0.0,1.0,0.0,0.0,1.0,0.375,0.33275,0.0,0.0
4,4,1,5,33,241,87,241,None,0.2,0.332,...,0.0,1.0,1.0,1.0,0.0,0.0,0.500,0.35775,0.0,0.0


# Q33B et Q27

In [ ]:
def select_conn(x) :
    if x == 1 or x == 2:
        return 1
    elif x == 3 :
        return 0
    else :
        return np.nan

def select_cont(x) :
    if x == 1 :
        return 1
    elif x == 2 or x == 3 :
        return 0
    else :
        return np.nan

raw_bva['Q33B'] = raw_bva['Q33'].astype(int)
bva['Q27A'] = raw_bva['Q27'].apply(select_conn)
bva['Q27B'] = raw_bva['Q27'].apply(select_cont)
bva['Q33BA'] = raw_bva['Q33B'].apply(select_conn)
bva['Q33BB'] = raw_bva['Q33B'].apply(select_cont)
bva['Q33B'] = raw_bva['Q33B']
bva['Q27'] = raw_bva['Q27']

# L2 & L3

In [ ]:
regle = {1 : "SAS FRET",
         2 : "SA GARES&CONNEXIONS",
         3 : "SA RESEAU",
         4 : "SA SNCF",
         5 : "SA VOYAGEURS",
         6 : "SFERIS",
         8 : "SANS AFFECTATION",
         9 : ''}

bva.loc[:, 'L2'] = bva['L2'].replace(regle).astype(str)

regle = {1: "AREP",
         3: "DG IDF",
         5: "DGA NUMERIQUE",
         7: "DIR GENERALEDELEGUEE STRATEGIE FINANCES",
         8: "DIR PRODUCTION",
         10: "DIR TECH",
         11: "DIR TECHNOLOGIES INNOVATION ET PROJETS GROUPE",
         14: "DIRECTION FRET SNCF",
         15: "DIRECTION GARES & CONNEXIONS",
         17: "DIRECTION GENERALE IMMOBILIER",
         18: "DIRECTION INDUSTRIELLE",
         20: "DIRECTION RESSOURCES HUMAINES GROUPE",
         21: "DIRECTION RISQUES AUDITS SECURITE SURETE",
         24: "OPTIM' SERVICES",
         26: "SANS AFFECTATION",
         27: "SIEGE",
         28: "SIEGE SNCF VOYAGEURS",
         31: "SURETE FERROVIAIRE",
         32: "TER",
         33: "TGV - INTERCITES",
         34: "TRANSILIEN",
         35: "ZP ATL",
         36: "ZP NEN",
         37: "ZP SE",
         38: "Sans réponse"}
bva.loc[:, 'L3'] = bva['L3'].replace(regle).astype(str)
bva['Q27'] = raw_bva['Q27']
bva['Q33B'] = raw_bva['Q33B']
bva.head()

C:\Users\0104785X\AppData\Local\Temp\ipykernel_21328\361534139.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['SA VOYAGEURS' 'SA VOYAGEURS' 'SA VOYAGEURS' ... 'SA VOYAGEURS'
 'SA VOYAGEURS' 'SA VOYAGEURS']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  bva.loc[:, 'L2'] = bva['L2'].replace(regle).astype(str)
C:\Users\0104785X\AppData\Local\Temp\ipykernel_21328\361534139.py:37: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['TER' 'TER' 'TRANSILIEN' ... 'TER' 'TER' 'TER']' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  bva.loc[:, 'L3'] = bva['L3'].replace(regle).astype(str)


,CLASSE,L1,L2,L3,ets_ref,ets1,ets2,ets3,Engagement standard,Engagement detail,...,Q9,QVT detail,Q29,Q10A,Q27A,Q27B,Q33BA,Q33BB,Q33B,Q27
0,4,1,SA VOYAGEURS,TER,317,317,None,None,0.0,0.198,...,0.000,0.13200,0.0,0.0,1.0,0.0,0.0,0.0,3,2
1,4,1,SA VOYAGEURS,TER,318,318,None,None,0.4,0.400,...,0.375,0.59450,0.0,0.0,1.0,0.0,1.0,0.0,2,2
2,1,1,SA VOYAGEURS,TRANSILIEN,187,68,187,None,1.0,1.000,...,1.000,0.86800,1.0,1.0,1.0,0.0,1.0,1.0,1,2
3,4,1,SA VOYAGEURS,TER,200,200,None,None,0.6,0.468,...,0.375,0.33275,0.0,0.0,0.0,0.0,NaN,NaN,4,3
4,4,1,SA VOYAGEURS,TGV - INTERCITES,241,87,241,None,0.2,0.332,...,0.500,0.35775,0.0,0.0,1.0,0.0,1.0,0.0,2,2


In [ ]:
bva.to_csv('../Données/Baro_SNCF_2023_Data_processed.csv', index=True, sep=';', na_rep='')